This files compute price from the strcutral model without Clusters. Consider the situation when 100 more houses are added to the market with different features. What should be the optimal price for this listings. 

I created 100 fake listing using row 101-200 arbitrarily. 

The algorithm computes the optimal prices for those 100 listings and also update the price for other lisitngs. The results show that the whole market prices are driven down by 8.5$ (8\%) when 100 more listings are added. 



In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns
import matplotlib.dates as mdates
from datetime import date
from sklearn.model_selection import train_test_split
import math


In [3]:
df = pd.read_csv('LA\March\listings.csv')


In [4]:
#data cleaning. Price data has $ sign. The following code extracts the number
df['price'] = df['price'].replace({r'\$':''},regex = True).replace({r',':''},regex = True).astype('float')
# delete the listings which are more expensive than $800
df1=df[df.price<800]

#get the subset of the data without missing 
df2= df1[~df1.beds.isna()&~df1.reviews_per_month.isna()&~df1.review_scores_accuracy.isna()&~df1.review_scores_checkin.isna()
         &~df1.review_scores_communication.isna()&~df1.review_scores_location.isna()&~df1.review_scores_value.isna()]
len(df2)/len(df1)

0.720346127144699

In [6]:
# for top 30 neighbourhood market # this can be replaced by hotspot

listtop30 = df2.neighbourhood_cleansed.value_counts().head(30).index.tolist()
df3=df2[df2["neighbourhood_cleansed"].isin(listtop30)]

df3 = df3.copy() 
# create market share from reviews
df3.loc[:,'share'] =df3['reviews_per_month']/(df3.groupby('neighbourhood_cleansed')['reviews_per_month'].transform('sum')*3)

# 2/3 is arbitrary, won't affect result much, create mean valuation of the listing
df3['mean_value'] =np.log(df3['share']/(2/3))


The following code computes IV results.

In [8]:
from statsmodels.sandbox.regression.gmm import IV2SLS
# instrument for price endogeneity
df3.loc[:,'IV'] = df3.groupby(['neighbourhood_cleansed'])['id'].transform('count')

Y=df3.mean_value
#with amenties


license_dummy=1*(~df3.license.isna()) 
# Fill NAN by "missing" string
df3[['host_response_time']] = df3[['host_response_time']].fillna('Missing_response')
host_response_dummy=pd.get_dummies(df3.host_response_time)
#room_type dummy
room_type_dummy=pd.get_dummies(df3.room_type)
super_host_dummy =1*(df3.host_is_superhost=="t")

amentity_count=df3.amenities.str.split().str.len()
wifi = 1*df3.amenities.str.lower().str.contains('wifi', regex=True) 
washer = 1*df3.amenities.str.lower().str.contains('washer', regex=True) 
dryer = 1*df3.amenities.str.lower().str.replace("hair dryer", " ").str.lower().str.contains('dryer', regex=True)
hair_dryer = 1*df3.amenities.str.lower().str.contains('hair dryer', regex=True) 
free_parking = 1*df3.amenities.str.lower().str.contains('free parking', regex=True) 
AC = 1*df3.amenities.str.lower().str.contains('air conditioning', regex=True) 
TV= 1*df3.amenities.str.lower().str.contains('tv', regex=True) 
hottub = 1*df3.amenities.str.lower().str.contains('hot tub', regex=True) 
coffee = 1*df3.amenities.str.lower().str.contains('coffee', regex=True) 
microwave = 1*df3.amenities.str.lower().str.contains('microwave', regex=True) 
pool = 1*df3.amenities.str.lower().str.contains('pool', regex=True) 
shampoo = 1*df3.amenities.str.lower().str.contains('shampoo', regex=True) 


Y=df3.mean_value
X=pd.concat([df3.price,df3.beds,license_dummy,
             room_type_dummy,super_host_dummy,df3.review_scores_rating,amentity_count,wifi,
             washer,dryer,hair_dryer,free_parking,AC,TV,hottub,coffee,microwave,pool,shampoo],axis=1)
exog_constant=X
#sm.add_constant(X)
endog = Y
instr_constant = sm.add_constant(pd.concat([df3.IV,df3.beds,license_dummy,
             room_type_dummy,super_host_dummy,df3.review_scores_rating,amentity_count,wifi,
             washer,dryer,hair_dryer,free_parking,AC,TV,hottub,coffee,microwave,pool,shampoo],axis=1))
endog_results = IV2SLS(endog, exog_constant, instrument = instr_constant).fit()
endog_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:             mean_value   R-squared:                     -46.063
Model:                         IV2SLS   Adj. R-squared:                -46.129
Method:                     Two Stage   F-statistic:                       nan
                        Least Squares   Prob (F-statistic):                nan
Date:                Mon, 29 May 2023                                         
Time:                        22:27:18                                         
No. Observations:               15052                                         
Df Residuals:                   15030                                         
Df Model:                          21                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
price                   -0.1038      0.016     -6.577      0.000      -0.135      -0.073
beds                     4.2210      0.634      6.662      0.000       2.979       5.463
license                  4.5189      0.677      6.679      0.000       3.193       5.845
Entire home/apt         -8.5537      1.171     -7.305      0.000     -10.849      -6.259
Hotel room             -10.4800      1.825     -5.742      0.000     -14.057      -6.903
Private room           -15.9352      1.561    -10.207      0.000     -18.995     -12.875
Shared room            -20.2491      2.235     -9.061      0.000     -24.630     -15.869
host_is_superhost        0.1972      0.194      1.017      0.309      -0.183       0.577
review_scores_rating     0.9700      0.240      4.040      0.000       0.499       1.441
amenities                0.0538      0.008      6.795      0.000       0.038       0.069
amenities                0.7586      0.820      0.926      0.355      -0.848       2.365
amenities                1.5064      0.436      3.454      0.001       0.652       2.361
amenities               -1.0006      0.317     -3.157      0.002      -1.622      -0.379
amenities                0.0909      0.262      0.347      0.728      -0.422       0.604
amenities                1.1493      0.241      4.761      0.000       0.676       1.622
amenities                2.1472      0.347      6.183      0.000       1.467       2.828
amenities                0.3073      0.315      0.975      0.330      -0.310       0.925
amenities                2.1924      0.440      4.982      0.000       1.330       3.055
amenities                0.7031      0.268      2.623      0.009       0.178       1.228
amenities               -3.4875      0.628     -5.557      0.000      -4.718      -2.257
amenities                0.3017      0.261      1.158      0.247      -0.209       0.812
amenities                0.9005      0.268      3.359      0.001       0.375       1.426
==============================================================================
Omnibus:                     5036.980   Durbin-Watson:                   1.938
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22806.090
Skew:                           1.579   Prob(JB):                         0.00
Kurtosis:                       8.137   Cond. No.                     7.03e+03
==============================================================================
"""

The following code computes markup based on IV regression results.

In [11]:
#markup 
markup =[]
df3 = df3.copy()
# number of the markets (hotspots)
M = df3['neighbourhood_cleansed'].nunique()
Group_market = df3.groupby('neighbourhood_cleansed')
#elaciticies
ela = [] 
for num, i in enumerate(df3['neighbourhood_cleansed'].unique()):
    submarket = Group_market.get_group(i)
    share_m = np.array(submarket.share)
    dsdp_m =  endog_results.params['price']*(np.diag(share_m)-np.outer(share_m,share_m)) #price derivatives
    
    #ela
    ela_m =dsdp_m*np.array(np.array(submarket.price).reshape(-1, 1) )/np.array(submarket.share)# that's how i compute elasticities  
    ela.append(ela_m)
    
    # markup
    host_id = submarket.host_id.to_numpy()
    T = host_id [:, np.newaxis] == host_id 
    markup.append(-np.dot(np.linalg.inv(T*dsdp_m),share_m))
     # assign markup back to dataFrame df3
    df3.loc[df3['neighbourhood_cleansed'] == i, 'markup'] = markup[num]

In [14]:
df3.markup.head() #markup for the first 10 listing in the first market

#profit margain in percentage

df3['margin_perc'] = df3.markup/df3.price
print("average markup per listing is $",str(round(df3.markup.mean(),2)) )
print("average rate of return is",str(round(df3.margin_perc.mean(),2)))
df3['mc'] = df3['price']-df3['markup']

average markup per listing is $ 9.66
average rate of return is 0.08


In [15]:
## compute counterfactual price if a new listing added. 
# create 100 new fake listing which is the as the row 101-200
new_row = df3.iloc[101:200,:]

print(new_row)
df3.reset_index(drop=True, inplace=True)
new_row.reset_index(drop=True, inplace=True)
df4 = pd.concat([df3,new_row], axis=0,ignore_index=True)

                     id                                      listing_url  \
101            46929502            https://www.airbnb.com/rooms/46929502   
102  736572730690948249  https://www.airbnb.com/rooms/736572730690948249   
103             8908966             https://www.airbnb.com/rooms/8908966   
104  781347569146558579  https://www.airbnb.com/rooms/781347569146558579   
105            37463168            https://www.airbnb.com/rooms/37463168   
..                  ...                                              ...   
195            21752969            https://www.airbnb.com/rooms/21752969   
196  825502584785160428  https://www.airbnb.com/rooms/825502584785160428   
197            30238317            https://www.airbnb.com/rooms/30238317   
198             8666764             https://www.airbnb.com/rooms/8666764   
199  620094545684358758  https://www.airbnb.com/rooms/620094545684358758   

          scrape_id last_scraped       source  \
101  20230307175359   2023-03-08  city

In [16]:
# compute the price 
tol = 1e-6


Group_market4 = df4.groupby('neighbourhood_cleansed')
for num, i in enumerate(df4['neighbourhood_cleansed'].unique()):
    submarket = Group_market4.get_group(i)
    new_price = np.array(submarket.price)
    diff=1
    while diff >tol: 
        mc= np.array(submarket.mc)
        new_share_m = np.array(submarket.mean_value)-endog_results.params['price']*(np.array(submarket.price)-new_price)
        dsdp_m = endog_results.params['price']*(np.diag(new_share_m)-np.outer(new_share_m,new_share_m)) #price derivatives

        #ela
        ela_m =dsdp_m*np.array(np.array(new_price).reshape(-1, 1) )/np.array(new_share_m)# that's how i compute elasticities  
        #ela.append(ela_m)

        # markup
        host_id = submarket.host_id.to_numpy()
        T = host_id [:, np.newaxis] == host_id 
        new_markup_m = -np.dot(np.linalg.inv(T*dsdp_m),new_share_m)
        old_price = new_price
        
        new_price =  mc+ new_markup_m
        diff= np.max(np.abs(old_price-new_price))
        #markup.append()
        
    # assign markup back to dataFrame df4
    df4.loc[df4['neighbourhood_cleansed'] == i, 'new markup'] = new_markup_m
    df4.loc[df4['neighbourhood_cleansed'] == i, 'new price'] = new_price


In [17]:
#print(df4['price'].head())
#print(df4['new price'].head())

price_change =df4['price']-df4['new price']
print(price_change.mean())

price_change_perc =(df4['price']-df4['new price'])/df4['new price']
print(price_change_perc.mean())

8.892071502200514
0.08785808597808208


 when 100 more listings added to the market. The market prices are driven down by 8.5$ (8\%)